<a href="https://colab.research.google.com/github/udaymallappa/ALADDIN/blob/master/DAGNN_UM_v2_jpeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
!git clone https://github.com/vthost/DAGNN.git
!mv DAGNN/ogb .
!mv DAGNN/src .

fatal: destination path 'DAGNN' already exists and is not an empty directory.
mv: cannot stat 'DAGNN/ogb': No such file or directory
mv: cannot stat 'DAGNN/src': No such file or directory


In [2]:
import torch; 

1.10.0+cu111


In [174]:
print(torch.__version__);

1.10.0+cu111


In [3]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 2.8 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 2.9 MB/s 
     |████████████████████████████████| 370 kB 5.1 MB/s 
     |████████████████████████████████| 482 kB 50.9 MB/s 
     |████████████████████████████████| 41 kB 554 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=4a2abe45a9e29971351165edf67cd65e536c2859546e49b48a85c559f9620fa6
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [4]:
!pip install timm

     |████████████████████████████████| 431 kB 5.2 MB/s 


In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import collections.abc as container_abcs

import torch.optim as optim

import math, numpy as np, scipy.sparse as sp
import torch.nn.init as init

from torch.autograd import Variable
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

from torch_scatter import scatter_add
import torch_geometric
from torch_geometric.nn.glob import *
from torch_geometric.nn.inits import uniform, glorot
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch import Tensor
from torch_geometric.typing import OptTensor
from torch_geometric.utils import softmax

import torch.utils.data
from torch.utils.data.dataloader import default_collate

from torch_geometric.data import Data, Batch
import collections.abc as container_abcs

from torchvision import transforms

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
def top_sort(edge_index, graph_size):

    node_ids = np.arange(graph_size, dtype=int)

    node_order = np.zeros(graph_size, dtype=int)
    unevaluated_nodes = np.ones(graph_size, dtype=bool)

    parent_nodes = edge_index[0]
    child_nodes = edge_index[1]

    counter = 0
    n = 0
    while unevaluated_nodes.any():
        print('In top sort index ', n)
        
        # Find which parent nodes have not been evaluated
        unevaluated_mask = unevaluated_nodes[parent_nodes]
        print('Number of parent nodes ', len(unevaluated_mask.nonzero()[0]))
        # Find the child nodes of unevaluated parents
        unready_children = child_nodes[unevaluated_mask]

        # Mark nodes that have not yet been evaluated
        # and which are not in the list of children with unevaluated parent nodes
        nodes_to_evaluate = unevaluated_nodes & ~np.isin(node_ids, unready_children)

        node_order[nodes_to_evaluate] = n
        unevaluated_nodes[nodes_to_evaluate] = False
        print('Number of unevaluated nodes ', len(nodes_to_evaluate.nonzero()[0]))
        counter += len(nodes_to_evaluate.nonzero()[0])
        n += 1
    print('Total nodes  = ', counter)
    return torch.from_numpy(node_order).long()

# to be able to use pyg's batch split everything into 1-dim tensors
def add_order_info_01(graph):
    print('Top sort Phase 1')
    l0 = top_sort(graph.edge_index, graph.num_nodes)
    ei2 = torch.LongTensor([list(graph.edge_index[1]), list(graph.edge_index[0])])
    print('Top sort Phase 2')
    l1 = top_sort(ei2, graph.num_nodes)
    ns = torch.LongTensor([i for i in range(graph.num_nodes)])

    graph.__setattr__("_bi_layer_idx0", l0)
    graph.__setattr__("_bi_layer_index0", ns)
    graph.__setattr__("_bi_layer_idx1", l1)
    graph.__setattr__("_bi_layer_index1", ns)

    assert_order(graph.edge_index, l0, ns)
    assert_order(ei2, l1, ns)

def assert_order(edge_index, o, ns):
    # already processed
    proc = []
    for i in range(max(o)+1):
        # nodes in position i in order
        l = o == i
        l = ns[l].tolist()
        for n in l:
            # predecessors
            ps = edge_index[0][edge_index[1] == n].tolist()
            for p in ps:
                assert p in proc
        proc += l


In [175]:
multicls_criterion = torch.nn.CrossEntropyLoss()

def train(model, device, loader, optimizer, args, evaluator):
    model.train()

    y_true = []
    y_pred = []
    loss_accum = 0
    print('Inside training module...')

    add_order_info_01(loader)#why this ? @chester
    batch = loader
    print('Batch === ' , batch)
    if batch:
        print('Before Pred')
        pred = model(batch)
        print('Pred done')
        optimizer.zero_grad()

        targ = torch.cat([batch[i].len_longest_path.to(device) for i in range(len(batch))], dim=0)

        loss = multicls_criterion(pred, targ.to(torch.long))  #batch.y.view(-1, ))
        loss.backward()
        if clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        loss_accum += loss.item()

        y_true.append(targ.view(-1,1).detach().cpu())
        y_pred.append(torch.argmax(pred.detach(), dim=1).view(-1, 1).cpu())

    y_true = torch.cat(y_true, dim=0).numpy()
    y_pred = torch.cat(y_pred, dim=0).numpy()
    #print(y_true)
    #print(y_pred)
    input_dict = {"y_true": y_true, "y_pred": y_pred}
    return loss_accum / (step + 1)

def eval(model, device, loader, evaluator):
    model.eval()

    y_true = []
    y_pred = []
    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        # one b(atch) per device
        #batch = [batch[i] for i in range(len(batch)) if not batch[i].x.shape[0] == 1]
        [add_order_info_01(b) for b in batch if not b.x.shape[0] == 1 and not b.batch[-1] == 0 ]
        batch = [b for b in batch if not b.x.shape[0] == 1]
        if batch:
            with torch.no_grad():
                pred = model(batch)
            # m = [b.len_longest_path.detach().cpu() for b in batch]
            # d = []
            # for b in batch:
            #     d += b.to_data_list()
            # d = [dd.node_depth.max().item() for dd in d]
            y_true1 = [batch[i].len_longest_path.view(-1,1).detach().cpu() for i in range(len(batch))]
            y_true += y_true1

            y_pred.append(torch.argmax(pred.detach(), dim = 1).view(-1,1).cpu())


    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0).view(y_pred.shape)
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()
    # print(y_true)
    # print(y_pred)
    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

def init_model(args, node_encoder, numclass=275):
  n = 1
  m = 1
  model = DAGNN(num_vocab=n, max_seq_len=m, emb_dim=emb_dim,
              hidden_dim=emb_dim, out_dim=None, encoder=node_encoder,
              w_edge_attr=ea, num_layers=dagnn_layers, bidirectional=bidir,
              agg=agg, mapper_bias=mapper_bias > 0,
              out_wx=out_wx > 0, out_pool_all=pool_all, out_pool=pool,
              dropout=dropout, num_class=numclass)
  return model

In [ ]:
from ogb.graphproppred.dataset import GraphPropPredDataset

In [ ]:
from ogb.graphproppred import Evaluator

In [ ]:
#from ogb.graphproppred.dataset_pyg import PygGraphPropPredDataset

In [12]:
import pandas as pd
import torch
from torch_geometric.data import Data
import os.path as osp
import numpy as np
from ogb.io.read_graph_raw import read_csv_graph_raw, read_csv_heterograph_raw, read_binary_graph_raw, read_binary_heterograph_raw
from tqdm import tqdm

#from src.utils_dag import add_order_info_01  # VT


def read_graph_pyg(raw_dir, add_inverse_edge = False, additional_node_files = [], additional_edge_files = [], binary = False):

    if binary:
        # npz
        graph_list = read_binary_graph_raw(raw_dir, add_inverse_edge)
    else:
        # csv
        graph_list = read_csv_graph_raw(raw_dir, add_inverse_edge, additional_node_files = additional_node_files, additional_edge_files = additional_edge_files)
    
    pyg_graph_list = []

    print('XXXX: Custom Code Alert XXXX Converting graphs into PyG objects...')

    gind = 0
    for graph in tqdm(graph_list):
        if gind > 100:
            break
        gind += 1
        print('Processing graph index : ', gind)
        print(graph.items())
        print(graph['edge_index'])
        print(graph['num_nodes'])


        g = Data()
        g.__num_nodes__ = graph['num_nodes']
        g.edge_index = torch.from_numpy(graph['edge_index'])

        del graph['num_nodes']
        del graph['edge_index']

        if graph['edge_feat'] is not None:
            g.edge_attr = torch.from_numpy(graph['edge_feat'])
            del graph['edge_feat']

        if graph['node_feat'] is not None:
            g.x = torch.from_numpy(graph['node_feat'])
            del graph['node_feat']

        for key in additional_node_files:
            g[key] = torch.from_numpy(graph[key])
            del graph[key]

        for key in additional_edge_files:
            g[key] = torch.from_numpy(graph[key])
            del graph[key]

        pyg_graph_list.append(g)

        add_order_info_01(g)  # DAGNN
        # length of longest path
        # layer ids start with 0 so max, gives actual path length and -1 is not necessary
        g.len_longest_path = float(torch.max(g._bi_layer_idx0).item())

    return pyg_graph_list


def read_heterograph_pyg(raw_dir, add_inverse_edge = False, additional_node_files = [], additional_edge_files = [], binary = False):

    if binary:
        # npz
        graph_list = read_binary_heterograph_raw(raw_dir, add_inverse_edge)
    else:
        # csv
        graph_list = read_csv_heterograph_raw(raw_dir, add_inverse_edge, additional_node_files = additional_node_files, additional_edge_files = additional_edge_files)

    pyg_graph_list = []

    print('Converting graphs into PyG objects...')

    for graph in tqdm(graph_list):
        g = Data()
        
        g.__num_nodes__ = graph['num_nodes_dict']
        g.num_nodes_dict = graph['num_nodes_dict']

        # add edge connectivity
        g.edge_index_dict = {}
        for triplet, edge_index in graph['edge_index_dict'].items():
            g.edge_index_dict[triplet] = torch.from_numpy(edge_index)

        del graph['edge_index_dict']

        if graph['edge_feat_dict'] is not None:
            g.edge_attr_dict = {}
            for triplet in graph['edge_feat_dict'].keys():
                g.edge_attr_dict[triplet] = torch.from_numpy(graph['edge_feat_dict'][triplet])

            del graph['edge_feat_dict']

        if graph['node_feat_dict'] is not None:
            g.x_dict = {}
            for nodetype in graph['node_feat_dict'].keys():
                g.x_dict[nodetype] = torch.from_numpy(graph['node_feat_dict'][nodetype])

            del graph['node_feat_dict']

        for key in additional_node_files:
            g[key] = {}
            for nodetype in graph[key].keys():
                g[key][nodetype] = torch.from_numpy(graph[key][nodetype])

            del graph[key]

        for key in additional_edge_files:
            g[key] = {}
            for triplet in graph[key].keys():
                g[key][triplet] = torch.from_numpy(graph[key][triplet])

            del graph[key]

        pyg_graph_list.append(g)

    return pyg_graph_list

In [13]:
from torch_geometric.data import InMemoryDataset
import pandas as pd
import shutil, os
import os.path as osp
import torch
import numpy as np
from ogb.utils.url import decide_download, download_url, extract_zip
#from ogb.io.read_graph_pyg import read_graph_pyg

In [46]:
### importing OGB


import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')
import pandas as pd
from tqdm import tqdm
import random

args = None

dataset="ogbg-code2"

filename = "name"
dir_data='./'

dir_results = 'DIR_RESULTS' 
dir_save = 'DIR_SAVED_MODELS'
gnn = 'dagnn'
device=0

lr=1e-3
clip=0.25

numclass=275
emb_dim=300
dagnn_layers=2
batch_size=160
num_workers = 0
bidir=1
ea=1
agg='attn_h'
dropout=0
pool_all=0
pool='max'
mapper_bias=1
out_wx=0

checkpoint=""
checkpointing=0

folds=1
epochs=1000
lay=5
patience=10

train_idx='train10' # use "" to run over the full dataset

device = torch.device("cuda:" + str(device)) if torch.cuda.is_available() else torch.device("cpu")

os.makedirs(dir_results, exist_ok=True)
os.makedirs(dir_save, exist_ok=True)

train_file = os.path.join(dir_results, filename + '_train.csv')
if not os.path.exists(train_file):
    with open(train_file, 'w') as f:
        f.write("fold,epoch,loss,train,valid,test\n")
res_file = os.path.join(dir_results, filename + '.csv')
if not os.path.exists(res_file):
    with open(res_file, 'w') as f:
        f.write("fold,epoch,bestv_train,bestv_valid,bestv_test\n")

### automatic dataloading and splitting
#dataset = PygGraphPropPredDataset(name=dataset, root="dataset" if dir_data is None else dir_data)
#dataset = PygNodePropPredDataset(name=dataset, root="dataset" if dir_data is None else dir_data)


In [132]:
import torch_geometric.data as Data
edge = np.loadtxt('/content/drive/MyDrive/dagnn_code/jpeg_edge_new.txt')
nx = np.loadtxt('/content/drive/MyDrive/dagnn_code/jpeg_encoder_node_x.txt')
ny = np.loadtxt('/content/drive/MyDrive/dagnn_code/jpeg_encoder_node_y.txt')


edge_tf = torch.tensor(edge.T, dtype=int)
nx_tf = torch.tensor(nx)
#ny_tf = torch.tensor(ny)
ny_tf = torch.randint(1,4, (nx_tf.shape[0],1))#dummy labels randomly created
D = Data.Data(x=nx_tf,edge_index=edge_tf, y=ny_tf)#, graph_size=nx.shape[0])

In [133]:
ny_tf

tensor([[3],
        [2],
        [3],
        ...,
        [3],
        [3],
        [1]])

In [113]:
#@title sample 12 node DAG
nx = np.random.rand(12,2)
ny = np.random.rand(12,1)
edge = np.random.rand(10,2)
edge[0] = [1,2]
edge[1] = [2,3]
edge[2] = [5,6]
edge[3] = [3,4]
edge[4] = [6,7]
edge[5] = [7,11]
edge[6] = [9,10]
edge[7] = [10,11]
edge[8] = [8,9]
edge[9] = [0,1]
edge_tf = torch.tensor(edge.T, dtype=int)
nx_tf = torch.tensor(nx)
ny_tf = torch.tensor(ny)
D = Data.Data(x=nx_tf,edge_index=edge_tf, y=ny_tf)#, graph_size=nx.shape[0])

In [176]:
ts = top_sort(D.edge_index, nx.shape[0])

In top sort index  0
Number of parent nodes  107045
Number of unevaluated nodes  5119
In top sort index  1
Number of parent nodes  95207
Number of unevaluated nodes  2668
In top sort index  2
Number of parent nodes  89265
Number of unevaluated nodes  1118
In top sort index  3
Number of parent nodes  84099
Number of unevaluated nodes  1947
In top sort index  4
Number of parent nodes  74804
Number of unevaluated nodes  4776
In top sort index  5
Number of parent nodes  62185
Number of unevaluated nodes  6541
In top sort index  6
Number of parent nodes  51012
Number of unevaluated nodes  5965
In top sort index  7
Number of parent nodes  41468
Number of unevaluated nodes  4429
In top sort index  8
Number of parent nodes  34541
Number of unevaluated nodes  3486
In top sort index  9
Number of parent nodes  28648
Number of unevaluated nodes  3169
In top sort index  10
Number of parent nodes  23582
Number of unevaluated nodes  2976
In top sort index  11
Number of parent nodes  19130
Number of u

In [110]:
#@title top sort for graphs (validation purpose)
#Python program to print topological sorting of a DAG
from collections import defaultdict
 
#Class to represent a graph
class Graph:
    def __init__(self,vertices):
        self.graph = defaultdict(list) #dictionary containing adjacency List
        self.V = vertices #No. of vertices
 
    # function to add an edge to graph
    def addEdge(self,u,v):
        self.graph[u].append(v)
 
    # A recursive function used by topologicalSort
    def topologicalSortUtil(self,v,visited,stack):
 
        # Mark the current node as visited.
        visited[v] = True
 
        # Recur for all the vertices adjacent to this vertex
        for i in self.graph[v]:
            if visited[i] == False:
                self.topologicalSortUtil(i,visited,stack)
 
        # Push current vertex to stack which stores result
        stack.insert(0,v)
 
    # The function to do Topological Sort. It uses recursive
    # topologicalSortUtil()
    def topologicalSort(self):
        # Mark all the vertices as not visited
        visited = [False]*self.V
        stack =[]
 
        # Call the recursive helper function to store Topological
        # Sort starting from all vertices one by one
        for i in range(self.V):
            if visited[i] == False:
                self.topologicalSortUtil(i,visited,stack)
 
        # Print contents of stack
        print(stack)
g= Graph(12)
g.addEdge(0,1)
g.addEdge(1,2)
g.addEdge(3,4)
g.addEdge(2,3)
g.addEdge(6,7)
g.addEdge(5,6)
g.addEdge(7,11)
g.addEdge(8,9)
g.addEdge(9,10)
g.addEdge(10,11)
g.topologicalSort()

In [ ]:
!mv /content/ogbg_code2 /content/drive/MyDrive/

In [ ]:
!cp -r /content/drive/MyDrive/ogbg_code2 /content/

In [126]:
from utils2 import augment_edge2

In [ ]:
split_idx = dataset.get_idx_split()

In [ ]:
split_idx

{'test': tensor([430793, 430794, 430795,  ..., 452738, 452739, 452740]),
 'train': tensor([     0,      1,      2,  ..., 407973, 407974, 407975]),
 'valid': tensor([407976, 407977, 407978,  ..., 430790, 430791, 430792])}

In [134]:
def init_model(args, node_encoder, numclass=275):
  n = 1
  m = 1
  model = DAGNN(num_vocab=n, max_seq_len=m, emb_dim=emb_dim,
              hidden_dim=emb_dim, out_dim=None, encoder=node_encoder,
              w_edge_attr=ea, num_layers=dagnn_layers, bidirectional=bidir,
              agg=agg, mapper_bias=mapper_bias > 0,
              out_wx=out_wx > 0, out_pool_all=pool_all, out_pool=pool,
              dropout=dropout, num_class=numclass)
  return model

In [135]:


path = os.path.abspath('.')
path = path[:path.rindex("/")] + "/../"
PATH = os.path.abspath(path)

# the latter are only default values used in config
DIR_DATA = os.path.join(PATH, 'data')
DIR_RESULTS = os.path.join(PATH, 'results')
DIR_SAVED_MODELS = os.path.join(PATH, 'saved_models')

NA_SUM = "add"
NA_MAX = "max"
NA_GATED_SUM = "gated_sum"
NA_SELF_ATTN_X = "self_attn_x"  # use xs of preds to compute weights, used to aggregate hs of preds
NA_SELF_ATTN_H = "self_attn_h"
NA_ATTN_X = "attn_x"  # use x and xs of preds to compute weights, used to aggregate hs of preds
NA_ATTN_H = "attn_h"  # use x and hs of preds
NA_MATTN_H = "mattn_h"  # use x and hs of preds

P_MEAN = "mean"
P_ADD = "add"  # do not use "sum" so that can be used to call tg pooling function
P_SUM = "sum"  # do not use "sum" so that can be used to call tg pooling function
P_MAX = "max"
P_ATTN = "attn"
EMB_POOLINGS = [P_MEAN, P_MAX, P_SUM]
POOLINGS = [P_MEAN, P_MAX, P_ATTN, P_ADD]
class DAGNN(nn.Module):

    def __init__(self, num_vocab, max_seq_len, emb_dim, hidden_dim, out_dim,
                 num_rels=2, w_edge_attr=True, num_layers=2, bidirectional=True, mapper_bias=True,  # bias only for DVAE simulation
                 agg_x=False, agg=NA_ATTN_H, out_wx=True, out_pool_all=True, out_pool=P_MAX, encoder=None, dropout=0.0,
                 word_vectors=None, emb_dims=[], activation=None, num_class=0, recurr=1):
        super().__init__()
        self.num_class = num_class
        self.num_vocab = num_vocab
        self.max_seq_len = max_seq_len

        if agg_x and hidden_dim < emb_dim:
            raise ValueError('Hidden dimension too small for input.')

        # configuration
        self.agg_x = agg_x  # use input states of predecessors instead of hidden ones
        self.agg_attn = "attn" in agg
        self.agg_attn_x = "_x" in agg
        self.bidirectional = bidirectional
        self.dirs = [0, 1] if bidirectional else [0]
        self.num_layers = num_layers
        self.out_wx = out_wx
        self.output_all = out_pool_all
        self.recurr = recurr

        # dimensions
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim
        self.out_hidden_dim = emb_dim * len(self.dirs) + self.hidden_dim * len(self.dirs) * num_layers if out_wx else self.hidden_dim * len(self.dirs) * num_layers  # USING UNIFY*len(self.dirs)
        # self.out_dim = out_dim  # not needed in OGB

        # initial embedding
        self.encoder = encoder if encoder is not None else init_encoder(word_vectors, emb_dims)

        # aggregate
        # agg_x makes only sense in first NN layer we could afterwards automatically use h? but postponing this...
        # (then add pred_dim term directly when looping over layers below)
        num_rels = num_rels if w_edge_attr else 1
        pred_dim = self.emb_dim if self.agg_x else self.hidden_dim
        attn_dim = self.emb_dim if "_x" in agg else self.hidden_dim
        if "self_attn" in agg:
            # it wouldn't make sense to perform attention based on h when aggregating x... so no hidden_dim needed
            self.node_aggr_0 = nn.ModuleList([
                SelfAttnConv(attn_dim, num_relations=num_rels) for _ in range(num_layers)])
            self.node_aggr_1 = nn.ModuleList([
                SelfAttnConv(attn_dim, num_relations=num_rels, reverse=True) for _ in range(num_layers)])
        elif "attn" in agg:
            op = MultAttnConv if "mattn" in agg else AttnConv
            self.node_aggr_0 = nn.ModuleList([
                op(self.emb_dim if l == 0 else attn_dim, pred_dim, num_relations=num_rels, attn_dim=attn_dim) for l in range(num_layers)])
            self.node_aggr_1 = nn.ModuleList([
                op(self.emb_dim if l == 0 else attn_dim, pred_dim, num_relations=num_rels, attn_dim=attn_dim, reverse=True) for l in range(num_layers)])
        elif agg == NA_GATED_SUM:
            self.node_aggr_0 = nn.ModuleList([
                GatedSumConv(pred_dim, num_rels, mapper_bias=mapper_bias) for _ in range(num_layers)])
            self.node_aggr_1 = nn.ModuleList([
                GatedSumConv(pred_dim, num_rels, mapper_bias=mapper_bias, reverse=True) for _ in range(num_layers)])
        else:
            node_aggr = AggConv(agg, num_rels, pred_dim)
            self.node_aggr_0 = self.node_aggr_1 = nn.ModuleList([node_aggr for _ in range(num_layers)])  # just to have same format

        # RNN
        if recurr:
            for i in self.dirs:
                self.__setattr__("cells_{}".format(i), nn.ModuleList(
                    [nn.GRUCell(emb_dim if l == 0 else self.hidden_dim, self.hidden_dim) for l in range(num_layers)]))
        else:
            for i in self.dirs:
                self.__setattr__("cells_{}".format(i), nn.ModuleList(
                    [nn.Linear((emb_dim if l == 0 else self.hidden_dim)+self.hidden_dim, self.hidden_dim) for l in range(num_layers)]))

        # readout
        if out_pool == P_ATTN:
            d = int(self.out_hidden_dim/2) if self.bidirectional and not self.output_all else self.out_hidden_dim
            self.self_attn_linear_out = torch.nn.Linear(d, 1)
            self._readout = self._out_nodes_self_attn
        else:
            self._readout = getattr(torch_geometric.nn, 'global_{}_pool'.format(out_pool))

        # output
        # self.out_norm = nn.LayerNorm(self.out_hidden_dim)
        self.dropout = nn.Dropout(dropout)
        # self.out_linear = torch.nn.Linear(self.out_hidden_dim, out_dim)
        # self.activation = init_activation(activation, out_dim)

        # OGB

        if self.num_class > 0:  # classification
            self.graph_pred_linear = torch.nn.Linear(self.out_hidden_dim, self.num_class)
        else:
            self.graph_pred_linear_list = torch.nn.ModuleList()
            if self.num_vocab == 1:  # regression
                self.graph_pred_linear_list.append(torch.nn.Sequential(
                    torch.nn.Linear(self.out_hidden_dim, self.num_vocab), torch.nn.ReLU()))
            else:
                for i in range(max_seq_len):
                    self.graph_pred_linear_list.append(torch.nn.Linear(self.out_hidden_dim, self.num_vocab))

    def _out_nodes_self_attn(self, h, batch):
        attn_weights = self.self_attn_linear_out(h)
        attn_weights = F.softmax(attn_weights, dim=-1)
        return global_add_pool(attn_weights * h, batch)

    def _get_output_nodes(self, G, reverse=0):
        if reverse:
            layer0 = G.bi_layer_index[0][0] == 0
            layer0 = G.bi_layer_index[0][1][layer0]
            return layer0
        layer0 = G.bi_layer_index[1][0] == 0
        layer0 = G.bi_layer_index[1][1][layer0]
        return layer0

    def forward(self, G):
        print('Inside Forward')
        # need to create these here since pyg's batching otherwise messes up the indices
        G.bi_layer_index = torch.stack([
            torch.stack([G._bi_layer_idx0, G._bi_layer_index0], dim=0),
            torch.stack([G._bi_layer_idx1, G._bi_layer_index1], dim=0)
        ], dim=0)
        print('Done with bi layer indexing')
        print(G.bi_layer_index)
        device = G.x.device
        num_nodes_batch = G.x.shape[0]
        num_layers_batch = max(G.bi_layer_index[0][0]).item() + 1

        G.x = self.encoder(G.x, G.node_depth.view(-1, ))

        G.h = [[torch.zeros(num_nodes_batch, self.hidden_dim).to(device)
                for _ in self.__getattr__("cells_{}".format(0))] for _ in self.dirs]

        for d in self.dirs:
            for l_idx in range(num_layers_batch):
                layer = G.bi_layer_index[d][0] == l_idx
                layer = G.bi_layer_index[d][1][layer]

                inp = G.x[layer]

                if l_idx > 0:  # no predecessors at first layer
                    le_idx = []
                    for n in layer:
                        ne_idx = G.edge_index[1-d] == n
                        le_idx += [ne_idx.nonzero().squeeze(-1)]
                    le_idx = torch.cat(le_idx, dim=-1)
                    lp_edge_index = G.edge_index[:, le_idx]

                    if self.agg_x:
                        # it wouldn't make sense to perform attention based on h when aggregating x... so no h needed
                        kwargs = {"h_attn": G.x, "h_attn_q": G.x} if self.agg_attn else {}  # just ignore query arg if self attn
                        node_agg = self.__getattr__("node_aggr_{}".format(d))[0]
                        ps_h = node_agg(G.x, lp_edge_index, edge_attr=G.edge_attr[le_idx], **kwargs)[layer]
                        # if we aggregate x...
                        s = ps_h.shape
                        if s[-1] < self.hidden_dim:
                            ps_h = torch.cat([ps_h, torch.zeros(s[0], self.hidden_dim-s[1])], dim=-1)
                        # print(G.x[lp_idx])
                        # print(ps_h)

                for i, cell in enumerate(self.__getattr__("cells_{}".format(d))):
                    if l_idx == 0:
                        ps_h = None if self.recurr else torch.zeros(inp.shape[0], self.hidden_dim).to(device)
                    elif not self.agg_x:
                        kwargs = {} if not self.agg_attn else \
                                    {"h_attn": G.x, "h_attn_q": G.x} if self.agg_attn_x else \
                                    {"h_attn": G.h[d][i], "h_attn_q": G.h[d][i-1] if i > 0 else G.x}  # just ignore query arg if self attn
                        node_agg = self.__getattr__("node_aggr_{}".format(d))[i]
                        ps_h = node_agg(G.h[d][i], lp_edge_index, edge_attr=G.edge_attr[le_idx], **kwargs)[layer]

                    inp = cell(inp, ps_h) if self.recurr else cell(torch.cat([inp, ps_h], dim=1))
                    G.h[d][i][layer] += inp

        if self.bidirectional and not self.output_all:
            index = self._get_output_nodes(G)
            h0 = torch.cat([G.x] + [G.h[0][l] for l in range(self.num_layers)], dim=-1) if self.out_wx else \
                torch.cat([G.h[0][l] for l in range(self.num_layers)], dim=-1)
            out0 = self._readout(h0[index], G.batch[index])
            index = self._get_output_nodes(G, reverse=1)
            h1 = torch.cat([G.x] + [G.h[1][l] for l in range(self.num_layers)], dim=-1) if self.out_wx else \
                torch.cat([G.h[1][l] for l in range(self.num_layers)], dim=-1)
            out1 = self._readout(h1[index], G.batch[index])
            out = torch.cat([out0, out1], dim=-1)
        else:
            G.h = torch.cat([G.x] + [G.h[d][l] for d in self.dirs for l in range(self.num_layers)], dim=-1) if self.out_wx else \
                torch.cat([G.h[d][l] for d in self.dirs for l in range(self.num_layers)], dim=-1) if self.bidirectional else \
                    torch.cat([G.h[0][l] for l in range(self.num_layers)], dim=-1)

            if not self.output_all:
                index = self._get_output_nodes(G)
                G.h, G.batch = G.h[index], G.batch[index]
            out = self._readout(G.h, G.batch)

        # out = self.out_linear(out)  #self.out_norm(out)
        out = self.dropout(out)
        # return self.activation(out).squeeze(-1)
        # return out

        if self.num_class > 0:
            return self.graph_pred_linear(out)

        pred_list = []
        for i in range(self.max_seq_len):
            pred_list.append(self.graph_pred_linear_list[i](out))
        return pred_list


def init_encoder(word_vectors, emb_dim):
    if word_vectors is not None:
        return nn.EmbeddingBag.from_pretrained(word_vectors, freeze=True, mode="sum")
    elif len(emb_dim) > 0:
        return nn.EmbeddingBag(emb_dim[0], emb_dim[1], mode="sum")
    return None

def init_param_emb(size, device):
    param = torch.zeros(size).to(device)
    glorot(param)
    # uniform(size, param)
    return param


class AggConv(MessagePassing):
    def __init__(self, agg, num_relations=1, emb_dim=0, reverse=False):
        super(AggConv, self).__init__(aggr=agg, flow='target_to_source' if reverse else 'source_to_target')

        if num_relations > 1:
            assert emb_dim > 0
            self.edge_encoder = torch.nn.Linear(num_relations, emb_dim)  # assuming num_relations one hot encoded
            self.wea = True
        else:
            self.wea = False

    def forward(self, x, edge_index, edge_attr=None, **kwargs):
        edge_embedding = self.edge_encoder(edge_attr) if self.wea else None
        return self.propagate(edge_index, x=x, edge_attr=edge_embedding)

    def message(self, x_j, edge_attr):
        return x_j + edge_attr if self.wea else x_j

    def update(self, aggr_out):
        return aggr_out


class GatedSumConv(MessagePassing):  # dvae needs outdim parameter
    def __init__(self, emb_dim, num_relations=1, mapper_bias=True, reverse=False):
        super(GatedSumConv, self).__init__(aggr='add', flow='target_to_source' if reverse else 'source_to_target')

        assert emb_dim > 0
        if num_relations > 1:
            self.wea = True
            self.edge_encoder = torch.nn.Linear(num_relations, emb_dim)
        else:
            self.wea = False
        self.mapper = nn.Linear(emb_dim, emb_dim, bias=mapper_bias)
        self.gate = nn.Sequential(nn.Linear(emb_dim, emb_dim), nn.Sigmoid())

    def forward(self, x, edge_index, edge_attr=None, **kwargs):
        edge_embedding = self.edge_encoder(edge_attr) if self.wea else None
        return self.propagate(edge_index, x=x, edge_attr=edge_embedding)

    def message(self, x_j, edge_attr):
        h_j = x_j + edge_attr if self.wea else x_j
        return self.gate(h_j) * self.mapper(h_j)

    def update(self, aggr_out):
        return aggr_out


class SelfAttnConv(MessagePassing):
    def __init__(self, emb_dim, attn_dim=0, num_relations=1, reverse=False):
        super(SelfAttnConv, self).__init__(aggr='add', flow='target_to_source' if reverse else 'source_to_target')

        assert emb_dim > 0
        attn_dim = attn_dim if attn_dim > 0 else emb_dim
        if num_relations > 1:
            self.wea = True
            self.edge_encoder = torch.nn.Linear(num_relations, attn_dim)
        else:
            self.wea = False
        self.attn_lin = nn.Linear(attn_dim, 1)

    # h_attn, edge_attr are optional
    def forward(self, h, edge_index, edge_attr=None, h_attn=None, **kwargs):
        edge_embedding = self.edge_encoder(edge_attr) if self.wea else None
        return self.propagate(edge_index, h=h, edge_attr=edge_embedding, h_attn=h_attn)

    def message(self, h_j, edge_attr, h_attn_j, index: Tensor, ptr: OptTensor, size_i: Optional[int]):
        h_attn = h_attn_j if h_attn_j is not None else h_j
        h_attn = h_attn + edge_attr if self.wea else h_attn
        # have to do part of this here instead of pre-computing a in forward because of missing edges in forward
        # in our dags there is not much overlap in one convolution step, so not much overhead here
        # and if attn transformation linear is applied in forward we'd have to consider full X/H matrices
        # which in our case can be a lot larger
        # BUT we could move it to forward similar to pyg GAT implementation
        # ie apply two different linear to each respectively X/H, edge_attrs which yield a scalar each
        # the in message only sum those up (to obtain a single scalar) and do softmax
        a_j = self.attn_lin(h_attn)
        a_j = softmax(a_j, index, ptr, size_i)
        t = h_j * a_j
        return t

    def update(self, aggr_out):
        return aggr_out

class AttnConv(MessagePassing):
    def __init__(self, attn_q_dim, emb_dim, attn_dim=0, num_relations=1, reverse=False):
        super(AttnConv, self).__init__(aggr='add', flow='target_to_source' if reverse else 'source_to_target')

        assert attn_q_dim > 0  # for us is not necessarily equal to attn dim at first RN layer
        assert emb_dim > 0
        attn_dim = attn_dim if attn_dim > 0 else emb_dim
        if num_relations > 1:
            self.wea = True
            self.edge_encoder = torch.nn.Linear(num_relations, attn_dim)
        else:
            self.wea = False
        self.attn_lin = nn.Linear(attn_q_dim + attn_dim, 1)

    # h_attn_q is needed; h_attn, edge_attr are optional (we just use kwargs to be able to switch node aggregator above)
    def forward(self, h, edge_index, h_attn_q=None, edge_attr=None, h_attn=None, **kwargs):
        edge_embedding = self.edge_encoder(edge_attr) if self.wea else None
        return self.propagate(edge_index, h_attn_q=h_attn_q, h=h, edge_attr=edge_embedding, h_attn=h_attn)

    def message(self, h_attn_q_i, h_j, edge_attr, h_attn_j, index: Tensor, ptr: OptTensor, size_i: Optional[int]):
        h_attn = h_attn_j if h_attn_j is not None else h_j
        h_attn = h_attn + edge_attr if self.wea else h_attn
        # see comment in above self attention why this is done here and not in forward
        a_j = self.attn_lin(torch.cat([h_attn_q_i, h_attn], dim=-1))
        a_j = softmax(a_j, index, ptr, size_i)
        t = h_j * a_j
        return t

    def update(self, aggr_out):
        return aggr_out


class MultAttnConv(MessagePassing):
    def __init__(self, attn_q_dim, emb_dim, attn_dim=0, num_relations=1, reverse=False):
        super(MultAttnConv, self).__init__(aggr='add', flow='target_to_source' if reverse else 'source_to_target')

        assert attn_q_dim > 0  # for us is not necessarily equal to attn dim at first RN layer
        assert emb_dim > 0
        attn_dim = attn_dim if attn_dim > 0 else emb_dim
        if num_relations > 1:
            self.wea = True
            self.edge_encoder = torch.nn.Linear(num_relations, attn_dim)
        else:
            self.wea = False
        self.attn_linl = nn.Linear(attn_q_dim, attn_q_dim)
        self.attn_linr = nn.Linear(attn_dim, attn_q_dim)

    # h_attn_q is needed; h_attn, edge_attr are optional (we just use kwargs to be able to switch node aggregator above)
    def forward(self, h, edge_index, h_attn_q=None, edge_attr=None, h_attn=None, **kwargs):
        edge_embedding = self.edge_encoder(edge_attr) if self.wea else None
        return self.propagate(edge_index, h_attn_q=h_attn_q, h=h, edge_attr=edge_embedding, h_attn=h_attn)

    def message(self, h_attn_q_i, h_j, edge_attr, h_attn_j, index: Tensor, ptr: OptTensor, size_i: Optional[int]):
        h_attn = h_attn_j if h_attn_j is not None else h_j
        h_attn = h_attn + edge_attr if self.wea else h_attn
        # see comment in above self attention why this is done here and not in forward
        a_j = torch.sum(self.attn_linl(h_attn_q_i) * self.attn_linr(h_attn), dim=1).unsqueeze(-1)
        a_j = softmax(a_j, index, ptr, size_i)
        t = h_j * a_j
        return t

    def update(self, aggr_out):
        return aggr_out

In [136]:

class DataParallel(torch.nn.DataParallel):
    r"""Implements data parallelism at the module level.

    This container parallelizes the application of the given :attr:`module` by
    splitting a list of :class:`torch_geometric.data.Data` objects and copying
    them as :class:`torch_geometric.data.Batch` objects to each device.
    In the forward pass, the module is replicated on each device, and each
    replica handles a portion of the input.
    During the backwards pass, gradients from each replica are summed into the
    original module.

    The batch size should be larger than the number of GPUs used.

    The parallelized :attr:`module` must have its parameters and buffers on
    :obj:`device_ids[0]`.

    .. note::

        You need to use the :class:`torch_geometric.data.DataListLoader` for
        this module.

    Args:
        module (Module): Module to be parallelized.
        device_ids (list of int or torch.device): CUDA devices.
            (default: all devices)
        output_device (int or torch.device): Device location of output.
            (default: :obj:`device_ids[0]`)
    """

    def __init__(self, module, device_ids=None, output_device=None):
        super(DataParallel, self).__init__(module, device_ids, output_device)
        self.src_device = torch.device("cuda:{}".format(self.device_ids[0])) if torch.cuda.is_available() else torch.device("cpu")  # VT allow cpu

    def forward(self, data_list):
        """"""
        if len(data_list) == 0:
            warnings.warn('DataParallel received an empty data list, which '
                          'may result in unexpected behaviour.')
            return None

        if not self.device_ids or len(self.device_ids) == 1:  # Fallback
            data = data_list[0].to(self.src_device)
            return self.module(data)

        for t in chain(self.module.parameters(), self.module.buffers()):
            if t.device != self.src_device:
                raise RuntimeError(
                    ('Module must have its parameters and buffers on device '
                     '{} but found one of them on device {}.').format(
                         self.src_device, t.device))

        inputs = self.scatter(data_list, self.device_ids)
        replicas = self.replicate(self.module, self.device_ids[:len(inputs)])
        outputs = self.parallel_apply(replicas, inputs, None)
        return self.gather(outputs, self.output_device)

    def scatter(self, data_list, device_ids):
        num_devices = min(len(device_ids), len(data_list))

        # count = torch.tensor([data.num_nodes for data in data_list])
        # cumsum = count.cumsum(0)
        # cumsum = torch.cat([cumsum.new_zeros(1), cumsum], dim=0)
        # device_id = num_devices * cumsum.to(torch.float) / cumsum[-1].item()
        # device_id = (device_id[:-1] + device_id[1:]) / 2.0
        # device_id = device_id.to(torch.long)  # round.
        # split = device_id.bincount().cumsum(0)
        # split = torch.cat([split.new_zeros(1), split], dim=0)
        # split = torch.unique(split, sorted=True)
        # split = split.tolist()

        return [
            data_list[i].to(
                torch.device('cuda:{}'.format(device_ids[i])))
            for i in range(num_devices)
        ]


In [ ]:
split_idx['train'] = r

tensor([     0,      1,      2,  ..., 407973, 407974, 407975])

In [ ]:
t = list(range(1,30))
v = list(range(31,60))
i = list(range(61,90))

In [ ]:
split_idx['train'] = torch.tensor(t, dtype=None)
split_idx['valid'] = torch.tensor(v, dtype=None)
split_idx['test'] = torch.tensor(i, dtype=None)

In [ ]:
from utils import augment_edge, encode_y_to_arr, decode_arr_to_seq
from utils2 import ASTNodeEncoder2
dataset.eval_metric = "acc"
dataset.task_type = "classification"

### set the transform function
# augment_edge: add next-token edge as well as inverse edges. add edge attributes.
# encode_y_to_arr: add y_arr to PyG data object, indicating the array representation of a sequence.
# DAGNN
augment = augment_edge2 if "dagnn" in gnn else augment_edge
dataset.transform = transforms.Compose([augment])  #, lambda data: encode_y_to_arr(data)])



nodetypes_mapping = pd.read_csv(os.path.join(dataset.root, 'mapping', 'typeidx2type.csv.gz'))
nodeattributes_mapping = pd.read_csv(os.path.join(dataset.root, 'mapping', 'attridx2attr.csv.gz'))
### Encoding node features into emb_dim vectors.
### The following three node features are used.
# 1. node type
# 2. node attribute
# 3. node depth
node_encoder = ASTNodeEncoder2(emb_dim, num_nodetypes = len(nodetypes_mapping['type']), num_nodeattributes = len(nodeattributes_mapping['attr']), max_depth = 20)


{'test': tensor([61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78,
         79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]),
 'train': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 'valid': tensor([31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
         49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])}

In [156]:
train_loader = DataLoader(D, batch_size=batch_size, shuffle=True)

In [157]:
D

Data(x=[57058, 19], edge_index=[2, 107045], y=[57058, 1])

In [163]:
train_loader.dataset.edge_index

tensor([[    0,     0,     0,  ..., 51989, 51990, 51990],
        [ 2603,  2604,  2046,  ..., 32022, 35422, 37010]])

In [177]:
# start training 
import collections.abc as container_abcs
int_classes = int
from torch._six import  string_classes
from dataloader import DataLoader
start_fold = 1
checkpoint_fn = ""
train_results, valid_results, test_results = [], [], []     # on fold level

for fold in range(start_fold, folds + 1):
    # fold-specific settings & data splits
    torch.manual_seed(fold)
    random.seed(fold)
    np.random.seed(fold)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(fold)
        torch.backends.cudnn.benchmark = True

    n_devices = torch.cuda.device_count() if torch.cuda.device_count() > 0 else 1
    train_loader = DataLoader(D, batch_size=batch_size, shuffle=True,
                              num_workers = num_workers)
    valid_loader = DataLoader(D, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers)
    test_loader = DataLoader(D, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers)

    start_epoch = 1

    # model etc.
    model = init_model(args, node_encoder=None)

    print("Let's use", torch.cuda.device_count(), "GPUs! -- DataParallel running also on CPU only")
    device_ids = list(range(torch.cuda.device_count())) if torch.cuda.device_count() > 0 else None
    model = DataParallel(model, device_ids)
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)

    # overwrite some settings
    if checkpointing and checkpoint:
        # signal that it has been used
        checkpoint = ""

        results, start_epoch, model, optimizer = load_checkpoint(checkpoint_fn, model, optimizer)
        train_results, valid_results, test_results, train_curve, valid_curve, test_curve = results
        start_epoch += 1
    else:
        valid_curve, test_curve, train_curve = [], [], []

    # start new epoch
    for epoch in range(start_epoch, epochs + 1):
        old_checkpoint_fn = checkpoint_fn
        checkpoint_fn = '%s.pt' % os.path.join(dir_save, filename + "_" + str(fold) + "_" + str(epoch))

        print("=====Fold {}, Epoch {}".format(fold, epoch))
        loss, train_perf = train(model, device, D, optimizer, args, evaluator=None)
        #valid_perf = eval(model, device, valid_loader, evaluator)
        #test_perf = eval(model, device, test_loader, evaluator)

        print({'Train': train_perf, 'Validation': valid_perf, 'Test': test_perf})
        #with open(train_file, 'a') as f:
        #    f.write("{},{},{:.4f},{:.4f},{:.4f},{:.4f}\n".format(fold, epoch, loss, train_perf[dataset.eval_metric], valid_perf[dataset.eval_metric], test_perf[dataset.eval_metric]))

        train_curve.append(train_perf[dataset.eval_metric])
        #valid_curve.append(valid_perf[dataset.eval_metric])
        #test_curve.append(test_perf[dataset.eval_metric])



    print('Finished training for fold {} !'.format(fold)+"*"*20)
    print('Best validation score: {}'.format(valid_curve[best_val_epoch]))
    print('Test score: {}'.format(test_curve[best_val_epoch]))

    with open(res_file, 'a') as f:
        results = [fold, best_val_epoch, train_curve[best_val_epoch], valid_curve[best_val_epoch],test_curve[best_val_epoch]]
        f.writelines(",".join([str(v) for v in results]) + "\n")

    train_results += [train_curve[best_val_epoch]]



Let's use 1 GPUs! -- DataParallel running also on CPU only
=====Fold 1, Epoch 1
Inside training module...
Top sort Phase 1
In top sort index  0
Number of parent nodes  107045
Number of unevaluated nodes  5119
In top sort index  1
Number of parent nodes  95207
Number of unevaluated nodes  2668
In top sort index  2
Number of parent nodes  89265
Number of unevaluated nodes  1118
In top sort index  3
Number of parent nodes  84099
Number of unevaluated nodes  1947
In top sort index  4
Number of parent nodes  74804
Number of unevaluated nodes  4776
In top sort index  5
Number of parent nodes  62185
Number of unevaluated nodes  6541
In top sort index  6
Number of parent nodes  51012
Number of unevaluated nodes  5965
In top sort index  7
Number of parent nodes  41468
Number of unevaluated nodes  4429
In top sort index  8
Number of parent nodes  34541
Number of unevaluated nodes  3486
In top sort index  9
Number of parent nodes  28648
Number of unevaluated nodes  3169
In top sort index  10
Numb

KeyError: ignored

     |████████████████████████████████| 370 kB 5.0 MB/s 
     |████████████████████████████████| 482 kB 63.1 MB/s 
     |████████████████████████████████| 41 kB 730 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.3-py3-none-any.whl size=581968 sha256=ca9ea01206d8c8320b6c068fa522c7b80b3797737d9f71a1e3d131223a11e43a
  Stored in directory: /root/.cache/pip/wheels/c3/2a/58/87ce0508964d4def1aafb92750c4f3ac77038efd1b9a89dcf5
Successfully built torch-geometric


In [ ]:
!pip install torch-sparse

     |████████████████████████████████| 43 kB 969 kB/s 
  Created wheel for torch-sparse: filename=torch_sparse-0.6.12-cp37-cp37m-linux_x86_64.whl size=1664876 sha256=19620db24027874ede83b309b615e0eb1f83a1d0b5b2ac43a53abf5df24abb20
  Stored in directory: /root/.cache/pip/wheels/fb/e2/2f/44956c61e3299573ffe12da9d1374c7576ca0c5fb1fe1ed38c
Successfully built torch-sparse


In [ ]:
from torch_geometric.utils import remove_self_loops

In [ ]:
import numpy
import torch
def top_sort(edge_index, graph_size):
    node_ids = numpy.arange(graph_size, dtype=int)
    node_order = numpy.zeros(graph_size, dtype=int)
    unevaluated_nodes = numpy.ones(graph_size, dtype=bool)
    parent_nodes = edge_index[0]
    child_nodes = edge_index[1]
    n = 0
    while unevaluated_nodes.any():
        # Find which parent nodes have not been evaluated
        unevaluated_mask = unevaluated_nodes[parent_nodes]
        # Find the child nodes of unevaluated parents
        unready_children = child_nodes[unevaluated_mask]
        # Mark nodes that have not yet been evaluated
        # and which are not in the list of children with unevaluated parent nodes
        nodes_to_evaluate = unevaluated_nodes & ~numpy.isin(node_ids, unready_children)
        node_order[nodes_to_evaluate] = n
        unevaluated_nodes[nodes_to_evaluate] = False
        print('Depth = ', n)
        n += 1
    return torch.from_numpy(node_order).long()

In [ ]:
edge = numpy.loadtxt('/content/drive/MyDrive/sizing/all_vt_new/train/MegaBoom_edges.txt')
x = numpy.loadtxt('/content/drive/MyDrive/sizing/all_vt_new/train/MegaBoom_node_x.txt')


In [ ]:
edge_tf = torch.Tensor(edge.T)
#x_tf = torch.Tensor(x)
ne = remove_self_loops(edge_tf)


In [ ]:
x.shape[0]

1639294

In [ ]:
edge_tf.shape

torch.Size([2, 4771206])

In [ ]:
ne1 = ne[0].int()

In [ ]:
ne1

tensor([[    0,     0,     0,  ..., 51989, 51990, 51990],
        [ 2603,  2604,  2046,  ..., 32022, 35422, 37010]], dtype=torch.int32)

In [ ]:
n_order = top_sort(ne[0].int(), x.shape[0])

Depth =  0
Depth =  1
Depth =  2
Depth =  3
Depth =  4
Depth =  5
Depth =  6
Depth =  7
Depth =  8
Depth =  9
Depth =  10
Depth =  11
Depth =  12
Depth =  13
Depth =  14
Depth =  15
Depth =  16
Depth =  17
Depth =  18
Depth =  19
Depth =  20
Depth =  21
Depth =  22
Depth =  23
Depth =  24
Depth =  25
Depth =  26
Depth =  27
Depth =  28
Depth =  29
Depth =  30
Depth =  31
Depth =  32
Depth =  33
Depth =  34
Depth =  35
Depth =  36
Depth =  37
Depth =  38
Depth =  39
Depth =  40
Depth =  41
Depth =  42
Depth =  43
Depth =  44
Depth =  45
Depth =  46
Depth =  47
Depth =  48
Depth =  49
Depth =  50
Depth =  51
Depth =  52
Depth =  53
Depth =  54
Depth =  55
Depth =  56
Depth =  57
Depth =  58
Depth =  59
Depth =  60
Depth =  61
Depth =  62
Depth =  63
Depth =  64
Depth =  65
Depth =  66
Depth =  67
Depth =  68
Depth =  69
Depth =  70
Depth =  71
Depth =  72
Depth =  73
Depth =  74
Depth =  75
Depth =  76
Depth =  77
Depth =  78
Depth =  79
Depth =  80
Depth =  81
Depth =  82
Depth =  83
De

In [ ]:
n_order

tensor([0, 0, 0,  ..., 0, 0, 0])